# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
# Load data from CSV generated from Part I to DataFrame
cityWeather = pd.read_csv('output_data/cities.csv', index_col=False)

# Rename index
cityWeather = cityWeather.set_index('Unnamed: 0')
cityWeather = cityWeather.rename_axis('index')

# Check length for quantity of data loaded
len(cityWeather)

571

### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude for locations and humidity for weight
locations = cityWeather[['Lat', 'Lng']].astype(float)
weight = cityWeather['Humidity'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=100, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [5]:
# Drop rows with NaN values
cityWeather = cityWeather.dropna()

# Ideal weather conditions to narrow down data
cityWeather2 = cityWeather[cityWeather['Max Temp'] <= 290]
cityWeather2 = cityWeather2[cityWeather2['Humidity'] < 75]
cityWeather2 = cityWeather2[cityWeather2['Wind Speed'] < 4]
cityWeather2 = cityWeather2[cityWeather2['Cloudiness'] < 10]
cityWeather2

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
index,,,,,,,,,
103,Anadyr,64.7500,177.4833,282.43,66,0,3.00,RU,1658696006
178,Abay,49.6311,72.8539,279.36,55,0,1.40,KZ,1658696032
221,Altay,47.8667,88.1167,288.37,43,0,2.03,CN,1658696049
300,Eucaliptus,-17.5833,-67.5167,287.36,17,0,1.80,BO,1658696082
326,Antofagasta,-23.6500,-70.4000,287.75,72,5,2.04,CL,1658696008
417,Gwanda,-20.9333,29.0000,289.43,38,0,2.43,ZW,1658696130
445,Comodoro Rivadavia,-45.8667,-67.5000,277.07,65,0,1.54,AR,1658696004
516,Omsk,55.0000,73.4000,286.09,71,0,0.00,RU,1658696121
517,Kuito,-12.3833,16.9333,288.90,34,2,1.00,AO,1658696170


### Hotel Map

In [6]:
# Creating hotel_df with narrowed down data
hotel_df = cityWeather2

# Add columns to store hotel data
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Lat'] = ""
hotel_df['Hotel Lon'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lon
index,,,,,,,,,,,,
103,Anadyr,64.7500,177.4833,282.43,66,0,3.00,RU,1658696006,,,
178,Abay,49.6311,72.8539,279.36,55,0,1.40,KZ,1658696032,,,
221,Altay,47.8667,88.1167,288.37,43,0,2.03,CN,1658696049,,,
300,Eucaliptus,-17.5833,-67.5167,287.36,17,0,1.80,BO,1658696082,,,
326,Antofagasta,-23.6500,-70.4000,287.75,72,5,2.04,CL,1658696008,,,
417,Gwanda,-20.9333,29.0000,289.43,38,0,2.43,ZW,1658696130,,,
445,Comodoro Rivadavia,-45.8667,-67.5000,277.07,65,0,1.54,AR,1658696004,,,
516,Omsk,55.0000,73.4000,286.09,71,0,0.00,RU,1658696121,,,
517,Kuito,-12.3833,16.9333,288.90,34,2,1.00,AO,1658696170,,,


In [7]:
# params dictionary for each iteration to retrieve data
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# For loop will identify nearby hotels using lat/lng of narrowed down cities
for column, row in hotel_df.iterrows():
    print(f"Retrieving Hotel Data for {row['City']}, {row['Country']}.")
    
    # Retrieve lat and lng from dataframe
    lat = row['Lat']
    lng = row['Lng']
    
    # Uses new location lat and lng during each iteration
    params["location"] = f"{lat},{lng}"
    
    # Make API call to retrieve data from Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params).json()
    
    # Checks for missing data using try-except and loads in NaN for any missing data
    try:
        print(f"Closest Hotel is {name_address['results'][0]['name']}")
        
        hotel_df.loc[column, 'Hotel Name'] = name_address['results'][0]['name']
        hotel_df.loc[column, 'Hotel Lat'] = name_address['results'][0]['geometry']['location']['lat']
        hotel_df.loc[column, 'Hotel Lon'] = name_address['results'][0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("No hotel found within radius. Skipping...")
        hotel_df.loc[column, 'Hotel Name'] = np.NaN
        hotel_df.loc[column, 'Hotel Lat'] = np.NaN
        hotel_df.loc[column, 'Hotel Lon'] = np.NaN
        
    print(f"Data processing for {row['City']}, {row['Country']} is complete.")
    print("-----------------------")

hotel_df

Retrieving Hotel Data for Anadyr, RU.
Closest Hotel is Anadyr
Data processing for Anadyr, RU is complete.
-----------------------
Retrieving Hotel Data for Abay, KZ.
Closest Hotel is Guest House on Abaya 144
Data processing for Abay, KZ is complete.
-----------------------
Retrieving Hotel Data for Altay, CN.
Closest Hotel is Xuedu Hotel
Data processing for Altay, CN is complete.
-----------------------
Retrieving Hotel Data for Eucaliptus, BO.
Closest Hotel is Hotel Serrano
Data processing for Eucaliptus, BO is complete.
-----------------------
Retrieving Hotel Data for Antofagasta, CL.
Closest Hotel is Hotel NH Antofagasta
Data processing for Antofagasta, CL is complete.
-----------------------
Retrieving Hotel Data for Gwanda, ZW.
Closest Hotel is Phumulani Hotel
Data processing for Gwanda, ZW is complete.
-----------------------
Retrieving Hotel Data for Comodoro Rivadavia, AR.
Closest Hotel is Lucania Palazzo Hotel
Data processing for Comodoro Rivadavia, AR is complete.
----------

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lon
index,,,,,,,,,,,,
103,Anadyr,64.7500,177.4833,282.43,66,0,3.00,RU,1658696006,Anadyr,64.73364,177.516873
178,Abay,49.6311,72.8539,279.36,55,0,1.40,KZ,1658696032,Guest House on Abaya 144,49.63092,72.865664
221,Altay,47.8667,88.1167,288.37,43,0,2.03,CN,1658696049,Xuedu Hotel,47.850277,88.140045
300,Eucaliptus,-17.5833,-67.5167,287.36,17,0,1.80,BO,1658696082,Hotel Serrano,-17.556535,-67.525382
326,Antofagasta,-23.6500,-70.4000,287.75,72,5,2.04,CL,1658696008,Hotel NH Antofagasta,-23.681189,-70.413978
417,Gwanda,-20.9333,29.0000,289.43,38,0,2.43,ZW,1658696130,Phumulani Hotel,-20.941575,29.013702
445,Comodoro Rivadavia,-45.8667,-67.5000,277.07,65,0,1.54,AR,1658696004,Lucania Palazzo Hotel,-45.863056,-67.476944
516,Omsk,55.0000,73.4000,286.09,71,0,0.00,RU,1658696121,ibis Sibir Omsk,54.980262,73.376997
517,Kuito,-12.3833,16.9333,288.90,34,2,1.00,AO,1658696170,iu Hotel Kuíto,-12.397799,16.940121


In [8]:
# Drops rows with NaN value
hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lon
index,,,,,,,,,,,,
103,Anadyr,64.7500,177.4833,282.43,66,0,3.00,RU,1658696006,Anadyr,64.73364,177.516873
178,Abay,49.6311,72.8539,279.36,55,0,1.40,KZ,1658696032,Guest House on Abaya 144,49.63092,72.865664
221,Altay,47.8667,88.1167,288.37,43,0,2.03,CN,1658696049,Xuedu Hotel,47.850277,88.140045
300,Eucaliptus,-17.5833,-67.5167,287.36,17,0,1.80,BO,1658696082,Hotel Serrano,-17.556535,-67.525382
326,Antofagasta,-23.6500,-70.4000,287.75,72,5,2.04,CL,1658696008,Hotel NH Antofagasta,-23.681189,-70.413978
417,Gwanda,-20.9333,29.0000,289.43,38,0,2.43,ZW,1658696130,Phumulani Hotel,-20.941575,29.013702
445,Comodoro Rivadavia,-45.8667,-67.5000,277.07,65,0,1.54,AR,1658696004,Lucania Palazzo Hotel,-45.863056,-67.476944
516,Omsk,55.0000,73.4000,286.09,71,0,0.00,RU,1658696121,ibis Sibir Omsk,54.980262,73.376997
517,Kuito,-12.3833,16.9333,288.90,34,2,1.00,AO,1658696170,iu Hotel Kuíto,-12.397799,16.940121


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lon"]]

In [11]:
weight = hotel_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=100, point_radius=1)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))